In [4]:
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
import os

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def check_fingerprint(wb, matr):
    ws = wb.worksheets[0]
    if int(ws["BB10"].value) == int(matr):
        return "OK"
    else:
        return "NOK"

def check_arrayformula(wb, matr):
    try:
        response = "        "
        ws = wb.worksheets[0]
        if ws.array_formulae != {}:
            response = response[:1] + "1" + response[2:]
        
        ws = wb.worksheets[1]
        if ws.array_formulae != {}:
            response = response[:3] + "2" + response[4:]

        ws = wb.worksheets[2]
        if ws.array_formulae != {}:
            response = response[:5] + "3" + response[6:]

        return response
    except:
        print("FAIL:", matr, "check Matrix_Formula")
        return "NOK"
        
def check_exercise_1_1(wb):
    ws = wb.worksheets[0]
    points = 0
    # Check formatting of cells A1:B2
    if ws["J4"].fill.start_color.index == "FF00B1AC" and ws["N1"].font.size == 16:
        points += 1
    return points

def check_exercise_1_2(wb):
    ws = wb.worksheets[0]
    points = 0
    # Check if T3 and AC3 contain a SUM or a +
    if "T3" in ws.array_formulae:
        formula = ws["T3"].value.text
    else:
        if ws["T3"].value != None:
            formula = ws["T3"].value
        else:
            formula = ""
    if "SUM" in formula or "+" in formula:
        points += 1

    if "AC3" in ws.array_formulae:
        formula = ws["AC3"].value.text
    else:
        if ws["AC3"].value != None:
            formula = ws["AC3"].value
        else:
            formula = ""
    if "SUM" in formula or "+" in formula:
        points += 1

    # Check if formula in D3 
    if "D3" in ws.array_formulae:
        formula = ws["D3"].value.text
    else:
        if ws["D3"].value != None:
            formula = ws["D3"].value
        else:
            formula = ""
    if "IFERROR" in formula:
        points += 1
    if "VLOOKUP" in formula and ("A3" in formula or "A$3" in formula):
        points += 1

    # Check formula in H3
    if "H3" in ws.array_formulae:
        formula = ws["H3"].value.text
    else:
        if ws["H3"].value != None:
            formula = ws["H3"].value
        else:
            formula = ""
    if formula != "":
        if "IF" in formula and "F3" in formula and ("K$4" in formula or "K4" in formula):
            points += 1
        if "VLOOKUP" in formula and ("G3" in formula or "G$3" in formula) and ("TRUE" in formula or "1" in formula):
            points += 1   

    return points


def check_exercise_2_1(wb):
    ws = wb.worksheets[1]
    points = 0
    # Check Numberformat in A13, E14 and L15
    if ws["A13"].number_format == r'"M-"0000':
        points += 1 
    if ".00" in ws["E14"].number_format and "€" in ws["E14"].number_format and \
        ".00" in ws["L15"].number_format and "€" in ws["L15"].number_format:
        points += 1

    return points

def check_exercise_2_2(wb):
    ws = wb.worksheets[1]
    points = 0

    # Check formula in K3
    if "K3" in ws.array_formulae:
        formula = ws["K3"].value.text
    else:
        if ws["K3"].value != None:
            formula = ws["K3"].value
        else:
            formula = ""
    if "IF" in formula and ("G3" in formula or "G$3" in formula) and "none" in formula:
        points += 1
    if ("E3" in formula or "E$3" in formula) and ("0.2" in formula or "20%" in formula) and ("0.5" in formula or "50%" in formula):
        points += 1

    # Check Formula in L3
    if "L3" in ws.array_formulae:
        formula = ws["L3"].value.text
    else:
        if ws["L3"].value != None:
            formula = ws["L3"].value
        else:
            formula = ""
    if ("IF" in formula and "AND" in formula) or formula.count("IF")>=2:
        points += 2
    if "500" in formula and "100" in formula and ("I3" in formula or "I$3" in formula) and ("H3" in formula or "H$3" in formula):
        points += 1

    # Check Formula in M3
    if "M3" in ws.array_formulae:
        formula = ws["M3"].value.text
    else:
        if ws["M3"].value != None:
            formula = ws["M3"].value
            formula = str(formula)
        else:
            formula = ""
    if formula.count("IF") >= 2:
        points += 2
    if ("0.5" in formula or "50%" in formula) and ("0.3" in formula or "30%" in formula):
        points += 2

    return points

def check_exercise_3_1(wb):
    ws = wb.worksheets[2]
    points = 0

    # Check formula in G4
    if "G4" in ws.array_formulae:
        formula = ws["G4"].value.text
    else:
        if ws["G4"].value != None:
            formula = ws["G4"].value
            formula = str(formula)
        else:
            formula = ""
    if "MID" in formula and "$A4" in formula and "G$3" in formula:
        points += 2

    # Check formula in P4
    if "P4" in ws.array_formulae:
        formula = ws["P4"].value.text
    else:
        if ws["P4"].value != None:
            formula = ws["P4"].value
            formula = str(formula)
        else:
            formula = ""
    if "SUMPRODUCT" in formula and "C4" in formula and "N$2" in formula:
        points += 2

    # Check formula in R4
    if "R4" in ws.array_formulae:
        formula = ws["R4"].value.text
    else:
        if ws["R4"].value != None:
            formula = ws["R4"].value
            formula = str(formula)
        else:
            formula = ""
    if ("IF" in formula or "MOD" in formula) and ("Q4" in formula or "Q$4" in formula):
        points += 2

    # Check T4
    if ws["T4"].value != None:
        if ws["T4"].value == 9782093850228:
            points += 2
        elif "=A" in str(ws["T4"].value):
            points += 2
        elif "INDEX" in str(ws["T4"].value):
            points += 4
            
    return points
    

def check_workbook(matr, wb, wb_val):
    points = [matr]
    points.append(check_fingerprint(wb, matr))
    points.append(check_arrayformula(wb, matr))
    points.append(check_exercise_1_1(wb))
    points.append(check_exercise_1_2(wb))
    points.append(check_exercise_2_1(wb))
    points.append(check_exercise_2_2(wb))
    points.append(check_exercise_3_1(wb))
    #points.append(check_exercise_1_5(wb, wb_val))
    
    return points



l_points = common.check_ExcelFiles_in_Abgaben(check_workbook)

for points in l_points:
    print(points)
    
header = ["MatrNr", "Fingerprint", "Array", "1 Format", "1 Formula", "2 Format", "2 Formula", "3 Formula"] 
common.result_into_file(l_points, header)


['~$3539743.xlsx']
[3539743, 'OK', '        ', 1, 6, 2, 6, 8]


In [ ]:
os.chdir(".")
print(os.getcwd())

In [22]:
x = "ssfdgs"
print(x.count("IF"))

0
